# Wide & Deep Learning for Recommender Systems

- 2016년 구글이 발표한 추천시스템 논문
- 먼저 살펴 볼 Memorization, Generalization 정의 살펴보자

### Memorization

- more topical and directly relevant to the items
- 자주 등장하는 아이템, 피처의 상관관계 등을 활용
- 예시) 마스크 많이 산 사람은 비슷한 마스크만 추천 받게됨(low diversity)

### Generalization

- imporve diversity of the recommendations
- Explore new feature combinations
- 다양한 새로운 피처들을 활용하여 추천
- 예시) 마스크를 많이 산 사람이라고 할지라도 가전제품에 관심이 있었다면 가전제품 도 추천 받음(diversity improve)
- generalization은 위 예시 같은 역할을 함

# 1. 배경

### 기존 추천 모델의 한계

<font size=4> 1. Generalized Linear model </font>

- 빈번하게 주어진 데이터에 특화된 모델로 새롭게 관측된(unseen) 데이터에 대해서 취약하다.
- unseen 데이터 활용할 경우 오버피팅 문제 발생

<font size=4> 2. Embedding based Model </font>

- FM, DNN 방법을 활용한 모델은 Generalization 특화 되지만
- Non-zero predction으로 섬세한 추천이 불가능하다.(ex:마스크 많이 산사람 더 세분화된 마스크 추천 받지 못하고 가전체품을 추천 받게됨)


<font size=4> 제안점 </font>

- jointly training feed-forward neural network and linear model 방법 제안
- Google에서 오픈소스 형태로 api까지 제공함

# 2. Overview

<img width='900' src='img/WideDeep/WideDeepOverview.png'>

- Memorization 과 Generalization을 모두 학습시키는 방법론 제안

<img width='900' src='img/WideDeep/workflow.png'>

- 이 논문에 눈여겨 볼 점은 Mlops 기반 workflow 제공했다는 점
- 학습과 속도를 모두 챙겼다


# 3. Objection Wide & Deep

## 1. Wide Component

<img width='900' src='img/WideDeep/widecomponent.png'>

- linear model를 모사하는 component
- y는 유저 행동 여부 (0 or 1)
- 두가지 종류를 input으로 넣을 수 있다.
- 1.raw input feature: 기존 데이터 모두 혹은 일부
- 2.cross product feature: 기존 컬럼들을 조합하여 새로운 컬럼으로 만든다.
- 이는 feature 간 interaction을 반영하기 위함이다.

<img width='900' src='img/WideDeep/wideinput.jpeg'>

- 위 그림처럼 install_app =  [A,B], impression_app = [A,C] 종류의 앱을 데이터가 있다고 할 때
- 모든 조합의 수는 9개가 된다. 이를 데이터프레임으로 표현해 두었다.
- interaction 컬럼을 만들고 install, impression컬럼이 모두 1일 때 interaction 컬럼의 row 값이 1이 된다.
- 단점으로는 0이 되는 pair는 학습이 불가능하다.

## 2. Deep Component

<img width='900' src='img/WideDeep/deepcomponent.png'>

- deep 모델은 wide와는 달리 interaction 데이터를 만들어 줄 필요가 없다.
- 왜냐하면 임베딩 공간에 interaction이 충분히 표현되기 때문이다.

<img width='700' src='img/WideDeep/deepinput.png'>

- 두 피처의 interaction은 multi-layer가 만든 non-linear한 공간에 표현이 된다.
- 하지만 매우 적게 등장하는 interaction 정보는 충분한 정보가 부족하기 때문에 표현이 안될 수 있다.
- 이는 희소한 데이터일 경우 전혀 관계없는 아이템이 추천될 수도 있다.


# 4. 학습 방법



<img width='700' src='img/WideDeep/train.png'>

### wide

- FTRL 기반 L1 규제를 적용한다.

### deep

- Adagrad를 적용
- 각 layer 마다 relu를 적용한다.

### Wide & deep

<img width='700' src='img/WideDeep/widedeepconcat.png'>

- wide와 deep 각각 다른 optimizer가 적용된다.
- wide와 deep에서 나온 output을 concat
- sigmoid를 적용하여 확률 값을 계산하여 backprop을 진행한다.
- 이때 특징적인 부분은 Wide 와 Deep이 동시에 학습이 진행된다는 점이다.
- 기존 앙상블 기법은 독립적을 모델들이 다른 loss에 대해서 학습이 진행되고 모델을 합친다면
- 같은 loss 기반으로 학습이 진행된다는 점이 큰 차이다. 이를 joint training라고 부른다.

# 5. 속도 관련 실험

<img width='500' src='img/WideDeep/threadlatency.png'>

- thread를 많이 이용할 수록 latency가 14까지 크게 줄어드는 것을 볼 수 있다.
- app 서비스까지 고려한 모델이다.

# 6. 결론

- Memorization(wide)와 Generalization(deep)을 결합한 모델을 제안하였다.
- linear model과 embedding model의 장점 조합
- 서비스 환경 구현까지 고려한 모델 설계이다.

# 7. 실습

- kmrd 데이터를 통해 실습을 진행
- kmrd 평점 데이터를 target으로 1~10점 분포 평점을 binary로 전처리 진행
- 9점 초과 1 이하 0으로 예측하도록 하였다.
- input 데이터로 장르, year 컬럼 데이터를 이용하였다.

# Wide & Deep Learning for Recommender System

- Google에서 App Store를 활용해서 발표한 논문([링크](https://arxiv.org/pdf/1606.07792.pdf))


## Google 공식 문서
- Google의 AI Blog([링크](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html))
- Google의 Tensorflow github([링크](https://github.com/tensorflow/tensorflow/blob/v2.4.0/tensorflow/python/keras/premade/wide_deep.py#L34-L219))
- TensorFlow v2.4 API
  - [tf.keras.experimental.WideDeepModel](https://www.tensorflow.org/api_docs/python/tf/keras/experimental/WideDeepModel?hl=en#methods_2)
  - [tf.estimator.DNNLinearCombinedClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNLinearCombinedClassifier)

## 함께볼만한 PyTorch Library
- [pytorch-widedeep](https://github.com/jrzaurin/pytorch-widedeep)

In [ ]:
# !pip install pytorch-widedeep

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## DataLoader

In [7]:
path = 'data/kmrd/kmr_dataset/datafile/kmrd-small'

In [8]:
df = pd.read_csv(os.path.join(path,'rates.csv'))
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1234, shuffle=True)

In [ ]:
train_df.shape

(112568, 4)

In [ ]:
train_df = train_df[:]

In [10]:
# Load all related dataframe
movies_df = pd.read_csv(os.path.join(path, 'movies.txt'), sep='\t', encoding='utf-8')
movies_df = movies_df.set_index('movie')

castings_df = pd.read_csv(os.path.join(path, 'castings.csv'), encoding='utf-8')
countries_df = pd.read_csv(os.path.join(path, 'countries.csv'), encoding='utf-8')
genres_df = pd.read_csv(os.path.join(path, 'genres.csv'), encoding='utf-8')

# Get genre information
genres = [(list(set(x['movie'].values))[0], '/'.join(x['genre'].values)) for index, x in genres_df.groupby('movie')]
combined_genres_df = pd.DataFrame(data=genres, columns=['movie', 'genres'])
combined_genres_df = combined_genres_df.set_index('movie')

# Get castings information
castings = [(list(set(x['movie'].values))[0], x['people'].values) for index, x in castings_df.groupby('movie')]
combined_castings_df = pd.DataFrame(data=castings, columns=['movie','people'])
combined_castings_df = combined_castings_df.set_index('movie')

# Get countries for movie information
countries = [(list(set(x['movie'].values))[0], ','.join(x['country'].values)) for index, x in countries_df.groupby('movie')]
combined_countries_df = pd.DataFrame(data=countries, columns=['movie', 'country'])
combined_countries_df = combined_countries_df.set_index('movie')

movies_df = pd.concat([movies_df, combined_genres_df, combined_castings_df, combined_countries_df], axis=1)

print(movies_df.shape)
movies_df.head()

(999, 7)


,title,title_eng,year,grade,genres,people,country
movie,,,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,드라마/멜로/로맨스,"[4374, 178, 3241, 47952, 47953, 19538, 18991, ...","이탈리아,프랑스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,SF/코미디,"[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]",미국
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,SF/코미디,"[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,...",미국
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,서부/SF/판타지/코미디,"[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1...",미국
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,판타지/모험/SF/액션,"[1007, 535, 215, 1236, 35]",미국


In [ ]:
movies_df.columns

Index(['title', 'title_eng', 'year', 'grade', 'genres', 'people', 'country'], dtype='object')

In [ ]:
dummy_genres_df = movies_df['genres'].str.get_dummies(sep='/')
train_genres_df = train_df['movie'].apply(lambda x: dummy_genres_df.loc[x])
train_genres_df.head()

,SF,가족,공포,느와르,다큐멘터리,드라마,로맨스,멜로,모험,뮤지컬,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
94390,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
dummy_grade_df = pd.get_dummies(movies_df['grade'], prefix='grade')
train_grade_df = train_df['movie'].apply(lambda x: dummy_grade_df.loc[x])
train_grade_df.head()

,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,grade_PG-13,grade_R,grade_전체 관람가,grade_청소년 관람불가
137023,1,0,0,0,0,0,0,0,0
92868,0,0,0,0,1,0,0,0,0
94390,1,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,0,0,1,0
80155,1,0,0,0,0,0,0,0,0


In [ ]:
train_df['year'] = train_df.apply(lambda x: movies_df.loc[x['movie']]['year'], axis=1)

In [ ]:
train_df = pd.concat([train_df, train_grade_df, train_genres_df], axis=1)
train_df.head()

,user,movie,rate,time,year,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,48423,10764,10,1212241560,1987.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,17307,10170,10,1122185220,1985.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
94390,18180,10048,10,1573403460,2016.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,1498,10001,9,1432684500,2013.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,12541,10022,10,1370458140,1980.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
wide_cols = list(dummy_genres_df.columns) + list(dummy_grade_df.columns)
wide_cols

['SF',
 '가족',
 '공포',
 '느와르',
 '다큐멘터리',
 '드라마',
 '로맨스',
 '멜로',
 '모험',
 '뮤지컬',
 '미스터리',
 '범죄',
 '서부',
 '서사',
 '스릴러',
 '애니메이션',
 '액션',
 '에로',
 '전쟁',
 '코미디',
 '판타지',
 'grade_12세 관람가',
 'grade_15세 관람가',
 'grade_G',
 'grade_NR',
 'grade_PG',
 'grade_PG-13',
 'grade_R',
 'grade_전체 관람가',
 'grade_청소년 관람불가']

In [ ]:
print(len(wide_cols))
print(wide_cols)

wide_cols = wide_cols[:3]

30
['SF', '가족', '공포', '느와르', '다큐멘터리', '드라마', '로맨스', '멜로', '모험', '뮤지컬', '미스터리', '범죄', '서부', '서사', '스릴러', '애니메이션', '액션', '에로', '전쟁', '코미디', '판타지', 'grade_12세 관람가', 'grade_15세 관람가', 'grade_G', 'grade_NR', 'grade_PG', 'grade_PG-13', 'grade_R', 'grade_전체 관람가', 'grade_청소년 관람불가']


In [ ]:
# wide_cols = ['genre', 'grade']
# cross_cols = [('genre', 'grade')]
wide_cols

['SF', '가족', '공포']

In [ ]:
import itertools
from itertools import product  
unique_combinations = list(list(zip(wide_cols, element)) 
                           for element in product(wide_cols, repeat = len(wide_cols))) 

print(unique_combinations)
cross_cols = [item for sublist in unique_combinations for item in sublist]
cross_cols = [x for x in cross_cols if x[0] != x[1]]
cross_cols = list(set(cross_cols))
print(cross_cols)

[[('SF', 'SF'), ('가족', 'SF'), ('공포', 'SF')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '가족')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '공포')], [('SF', 'SF'), ('가족', '가족'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '가족'), ('공포', '가족')], [('SF', 'SF'), ('가족', '가족'), ('공포', '공포')], [('SF', 'SF'), ('가족', '공포'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '공포'), ('공포', '가족')], [('SF', 'SF'), ('가족', '공포'), ('공포', '공포')], [('SF', '가족'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '가족'), ('가족', 'SF'), ('공포', '가족')], [('SF', '가족'), ('가족', 'SF'), ('공포', '공포')], [('SF', '가족'), ('가족', '가족'), ('공포', 'SF')], [('SF', '가족'), ('가족', '가족'), ('공포', '가족')], [('SF', '가족'), ('가족', '가족'), ('공포', '공포')], [('SF', '가족'), ('가족', '공포'), ('공포', 'SF')], [('SF', '가족'), ('가족', '공포'), ('공포', '가족')], [('SF', '가족'), ('가족', '공포'), ('공포', '공포')], [('SF', '공포'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '공포'), ('가족', 'SF'), ('공포', '가족')], [('SF', '공포'), ('가족', 'SF'), ('공포', '공포')], [('SF', '공포'), ('가족', '가족'), ('공포', 'SF')], [('SF', '공포'), ('가족', '가족'), ('

In [ ]:
# embed_cols = [('genre', 16),('grade', 16)]
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['year']

print(embed_cols)
print(continuous_cols)

[('가족', 16), ('SF', 16), ('공포', 16)]
['year']


In [ ]:
target = train_df['rate'].apply(lambda x: 1 if x > 9 else 0).values

## Wide & Deep

In [ ]:
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep.datasets import load_adult

### Wide Component

In [ ]:

wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = wide_preprocessor.fit_transform(train_df)
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)


In [ ]:
X_wide.size

1013112

In [ ]:
wide

Wide(
  (wide_linear): Embedding(31, 1, padding_idx=0)
)

### Deep Component

In [ ]:
tab_preprocessor = TabPreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols) 
X_deep = tab_preprocessor.fit_transform(train_df)
deep_mlp = TabMlp(
    mlp_hidden_dims=[64, 32],
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    continuous_cols=continuous_cols,
)

In [ ]:
deep_mlp

TabMlp(
  (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
    (cat_embed): DiffSizeCatEmbeddings(
      (embed_layers): ModuleDict(
        (emb_layer_가족): Embedding(3, 16, padding_idx=0)
        (emb_layer_SF): Embedding(3, 16, padding_idx=0)
        (emb_layer_공포): Embedding(3, 16, padding_idx=0)
      )
      (embedding_dropout): Dropout(p=0.1, inplace=False)
    )
    (cont_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tab_mlp): MLP(
    (mlp): Sequential(
      (dense_layer_0): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=49, out_features=64, bias=True)
        (2): ReLU(inplace=True)
      )
      (dense_layer_1): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=64, out_features=32, bias=True)
        (2): ReLU(inplace=True)
      )
    )
  )
)

### Build and Train

In [ ]:
model = WideDeep(wide=wide, deep_mlp=deep_mlp)

# train and validate
trainer = Trainer(model, objective="binary", metrics=[Accuracy])
trainer.fit(
    X_wide=X_wide,
    X_deep=X_deep,
    target=target,
    n_epochs=5,
    batch_size=256,
    val_split=0.1
)


valid: 100%|██████████| 44/44 [00:00<00:00, 313.56it/s, loss=0.63, metrics={'acc': 0.6709}] 


In [ ]:
X_deep.shape

(112568, 4)

In [ ]:
X_wide.shape

(112568, 9)